In [1]:
pip install tokopaedi

Note: you may need to restart the kernel to use updated packages.


In [17]:
from tokopaedi import get_reviews
import json

url = "https://www.tokopedia.com/liger-official-store/liger-handsfree-headset-earphone-l-10-metal-stereo-bass-biru-1731543200578176211"

all_reviews = []
target_count = 1000
page = 1

while len(all_reviews) < target_count:
    # Ambil review per halaman
    reviews = get_reviews(url=url, page=page, debug=True)
    
    if not reviews:  # Kalau sudah tidak ada review
        break
    
    all_reviews.extend(reviews)
    page += 1

all_reviews = all_reviews[:target_count]

# Rapikan ke JSON
reviews_json = [r.json() for r in all_reviews]

print(json.dumps(reviews_json, indent=2, ensure_ascii=False))


2025-08-29 12:58 - REVIEW - 470760771 - Barangnya original 👍. Suaranya jernih. D...
2025-08-29 12:58 - REVIEW - 457477791 - mantull.. 👍👍👍pake banget bass nya kuence...
2025-08-29 12:58 - REVIEW - 688491804 - desain, kualitas bahan dan suara abal ya...
2025-08-29 12:58 - REVIEW - 444733835 - berfungsi dengan baik, hanya produk ada ...
2025-08-29 12:58 - REVIEW - 485159499 - bagusbeefungsi dengan baikBarangnya orig...
2025-08-29 12:58 - REVIEW - 491474306 - Barangnya original 👍. Ada garansi resmi....
2025-08-29 12:58 - REVIEW - 458273007 - sesuai lah ya sama harga, bassnya kurang...
2025-08-29 12:58 - REVIEW - 879573481 - Suara nya yg sebelah ngebass, yg sebelah...
2025-08-29 12:58 - REVIEW - 494344182 - Suaranya jernih. Mantap dahh pokoknya 👍👍...
2025-08-29 12:58 - REVIEW - 682761127 - earplug sebelah kanan gak ada karet nya,...
2025-08-29 12:58 - REVIEW - 677304820 - Mantap. Murah. Berfungsi dengan baik....
2025-08-29 12:58 - REVIEW - 460996055 - Yaa.. lihat komen orang-orang berarti s

[
  {
    "feedback_id": 470760771,
    "variant_name": "Hitam",
    "message": "Barangnya original 👍. Suaranya jernih. Desainnya modern 😍. Kabelnya kuat. Bass-nya mantap.",
    "rating": 5.0,
    "review_age": "Lebih dari 1 tahun lalu",
    "user_full_name": "Franklin",
    "user_url": "https://www.tokopedia.com/people/44068819",
    "response_message": "Terima kasih telah berbelanja di LIGER OFFICIAL STORE. Bagikan link toko kami https://www.tokopedia.com/liger-official kepada teman-teman Anda dan favoritkan Toko kami untuk terus update mengenai stok dan produk terbaru",
    "response_created_text": "Lebih dari 1 tahun lalu",
    "images": [
      "https://images.tokopedia.net/img/cache/600/bjFkPX/2021/9/6/7c40422d-6c6f-4418-9d4c-312edcaa7c31.jpg",
      "https://images.tokopedia.net/img/cache/600/bjFkPX/2021/9/6/79824044-d3e2-41b6-b226-722ae37ccc87.jpg",
      "https://images.tokopedia.net/img/cache/600/bjFkPX/2021/9/6/a4ad8a07-0082-448d-9937-3a035e9d2923.jpg"
    ],
    "videos": [